In [1]:
%load_ext cudf.pandas

# Reading and writing files

One of Xarray's most widely used features is its ability to [read from and write
to a variety of data formats](https://docs.xarray.dev/en/stable/user-guide/io.html). 
For example, Xarray can read the following formats using `open_dataset`/`open_mfdataset`:

- [NetCDF](https://www.unidata.ucar.edu/software/netcdf/)
- [Zarr](https://zarr.readthedocs.io/en/stable/)

Support for additional formats is possible using external packages
- [GRIB](https://en.wikipedia.org/wiki/GRIB) using the [cfgrib](https://github.com/ecmwf/cfgrib) package
- [GeoTIFF](https://gdal.org/drivers/raster/gtiff.html) /
  [GDAL rasters](https://svn.osgeo.org/gdal/tags/gdal_1_2_5/frmts/formats_list.html)
  using the [rioxarray package](https://corteva.github.io/rioxarray/stable/)

<img src="https://www.unidata.ucar.edu/images/logos/netcdf-400x400.png" align="right" width="20%">

## NetCDF

The recommended way to store xarray data structures is NetCDF, which is a binary
file format for self-described datasets that originated in the geosciences.
Xarray is based on the netCDF data model, so netCDF files on disk directly
correspond to Dataset objects.

Xarray reads and writes to NetCDF files using the `open_dataset` /
`open_dataarray` functions and the `to_netcdf` method.

Let's first create some datasets and write them to disk using `to_netcdf`, which
takes the path we want to write to:


In [2]:
%%cudf.pandas.profile

import numpy as np
import xarray as xr

# Ensure random arrays are the same each time
np.random.seed(0)

                                                                                              
                                  Total time elapsed: 1.552 seconds                           
                                0 GPU function calls in 0.000 seconds                         
                                0 CPU function calls in 0.000 seconds                         
                                                                                              
                                                Stats                                         
                                                                                              
┏━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┓
┃ Function ┃ GPU ncalls ┃ GPU cumtime ┃ GPU percall ┃ CPU ncalls ┃ CPU cumtime ┃ CPU percall ┃
┡━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━┩
└──────────┴────────────┴─────────────┴─────────────┴────────────┴─────────────┴─────────────┘

The constructor of `Dataset` takes three parameters:

- `data_vars`: dict-like mapping names to values. Values are either `DataArray` objects
  or defined with tuples consisting of of dimension names and arrays.
- `coords`: same as for `DataArray`
- `attrs`: same as for `DataArray`

In [3]:
%%cudf.pandas.profile

ds1 = xr.Dataset(
    data_vars={
        "a": (("x", "y"), np.random.randn(4, 2)),
        "b": (("z", "x"), np.random.randn(6, 4)),
    },
    coords={
        "x": np.arange(4),
        "y": np.arange(-2, 0),
        "z": np.arange(-3, 3),
    },
)
ds2 = xr.Dataset(
    data_vars={
        "a": (("x", "y"), np.random.randn(7, 3)),
        "b": (("z", "x"), np.random.randn(2, 7)),
    },
    coords={
        "x": np.arange(6, 13),
        "y": np.arange(3),
        "z": np.arange(3, 5),
    },
)

# write datasets
ds1.to_netcdf("ds1.nc")
ds2.to_netcdf("ds2.nc")

# write dataarray
ds1.a.to_netcdf("da1.nc")

                                                                                                            
                                         Total time elapsed: 3.243 seconds                                  
                                      194 GPU function calls in 1.549 seconds                               
                                       0 CPU function calls in 0.000 seconds                                
                                                                                                            
                                                       Stats                                                
                                                                                                            
┏━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┓
┃ Function               ┃ GPU ncalls ┃ GPU cumtime ┃ GPU percall ┃ CPU ncalls ┃ CPU cumtime ┃ CPU percall ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━┩
│ Index                  │ 6          │ 1.490       │ 0.248       │ 0          │ 0.000       │ 0.000       │
│ Int64Index.__len__     │ 162        │ 0.047       │ 0.000       │ 0          │ 0.000       │ 0.000       │
│ Int64Index.copy        │ 6          │ 0.002       │ 0.000       │ 0          │ 0.000       │ 0.000       │
│ Int64Index.__setattr__ │ 12         │ 0.005       │ 0.000       │ 0          │ 0.000       │ 0.000       │
│ Int64Index.set_names   │ 8          │ 0.004       │ 0.001       │ 0          │ 0.000       │ 0.000       │
└────────────────────────┴────────────┴─────────────┴─────────────┴────────────┴─────────────┴─────────────┘

Reading those files is just as simple:


In [4]:
%%cudf.pandas.profile

xr.open_dataset("ds1.nc")

<xarray.Dataset>
Dimensions:  (x: 4, y: 2, z: 6)
Coordinates:
  * x        (x) int64 0 1 2 3
  * y        (y) int64 -2 -1
  * z        (z) int64 -3 -2 -1 0 1 2
Data variables:
    a        (x, y) float64 ...
    b        (z, x) float64 ...

                                                                                                            
                                         Total time elapsed: 1.501 seconds                                  
                                      288 GPU function calls in 0.108 seconds                               
                                       10 CPU function calls in 0.005 seconds                               
                                                                                                            
                                                       Stats                                                
                                                                                                            
┏━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┓
┃ Function               ┃ GPU ncalls ┃ GPU cumtime ┃ GPU percall ┃ CPU ncalls ┃ CPU cumtime ┃ CPU percall ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━┩
│ isna                   │ 0          │ 0.000       │ 0.000       │ 10         │ 0.005       │ 0.001       │
│ Index                  │ 3          │ 0.003       │ 0.001       │ 0          │ 0.000       │ 0.000       │
│ Int64Index.__len__     │ 252        │ 0.073       │ 0.000       │ 0          │ 0.000       │ 0.000       │
│ Int64Index.copy        │ 6          │ 0.002       │ 0.000       │ 0          │ 0.000       │ 0.000       │
│ Int64Index.__setattr__ │ 6          │ 0.002       │ 0.000       │ 0          │ 0.000       │ 0.000       │
│ Int64Index.__getitem__ │ 18         │ 0.015       │ 0.001       │ 0          │ 0.000       │ 0.000       │
│ Int64Index.__repr__    │ 3          │ 0.013       │ 0.004       │ 0          │ 0.000       │ 0.000       │
└────────────────────────┴────────────┴─────────────┴─────────────┴────────────┴─────────────┴─────────────┘

Not all pandas operations ran on the GPU. The following functions required CPU fallback:

- isna

To request GPU support for any of these functions, please file a Github issue here: 
]8;id=181914;https://github.com/rapidsai/cudf/issues/new?assignees=&labels=%3F+-+Needs+Triage%2C+feature+request&projects=&template=pandas_function_request.md&title=%5BFEA%5D\https://github.com/rapidsai/cudf/issues/new/choose]8;;\.

In [5]:
%%cudf.pandas.profile

xr.open_dataarray("da1.nc")

<xarray.DataArray 'a' (x: 4, y: 2)>
[8 values with dtype=float64]
Coordinates:
  * x        (x) int64 0 1 2 3
  * y        (y) int64 -2 -1

                                                                                                            
                                         Total time elapsed: 0.233 seconds                                  
                                      191 GPU function calls in 0.070 seconds                               
                                       6 CPU function calls in 0.003 seconds                                
                                                                                                            
                                                       Stats                                                
                                                                                                            
┏━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┓
┃ Function               ┃ GPU ncalls ┃ GPU cumtime ┃ GPU percall ┃ CPU ncalls ┃ CPU cumtime ┃ CPU percall ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━┩
│ isna                   │ 0          │ 0.000       │ 0.000       │ 6          │ 0.003       │ 0.001       │
│ Index                  │ 2          │ 0.002       │ 0.001       │ 0          │ 0.000       │ 0.000       │
│ Int64Index.__len__     │ 167        │ 0.048       │ 0.000       │ 0          │ 0.000       │ 0.000       │
│ Int64Index.copy        │ 4          │ 0.001       │ 0.000       │ 0          │ 0.000       │ 0.000       │
│ Int64Index.__setattr__ │ 4          │ 0.002       │ 0.000       │ 0          │ 0.000       │ 0.000       │
│ Int64Index.__getitem__ │ 12         │ 0.010       │ 0.001       │ 0          │ 0.000       │ 0.000       │
│ Int64Index.__repr__    │ 2          │ 0.007       │ 0.004       │ 0          │ 0.000       │ 0.000       │
└────────────────────────┴────────────┴─────────────┴─────────────┴────────────┴─────────────┴─────────────┘

Not all pandas operations ran on the GPU. The following functions required CPU fallback:

- isna

To request GPU support for any of these functions, please file a Github issue here: 
]8;id=44329;https://github.com/rapidsai/cudf/issues/new?assignees=&labels=%3F+-+Needs+Triage%2C+feature+request&projects=&template=pandas_function_request.md&title=%5BFEA%5D\https://github.com/rapidsai/cudf/issues/new/choose]8;;\.

<img src="https://zarr.readthedocs.io/en/stable/_static/logo1.png" align="right" width="20%">


## Zarr

[Zarr](https://zarr.readthedocs.io/en/stable/) is a Python package and data
format providing an implementation of chunked, compressed, N-dimensional arrays.
Zarr has the ability to store arrays in a range of ways, including in memory, in
files, and in cloud-based object storage such as Amazon S3 and Google Cloud
Storage. Xarray’s Zarr backend allows xarray to leverage these capabilities.

Zarr files can be written with:


In [6]:
%%cudf.pandas.profile

ds1.to_zarr("ds1.zarr", mode="w")

                                                                                                          
                                        Total time elapsed: 0.493 seconds                                 
                                      36 GPU function calls in 0.011 seconds                              
                                      0 CPU function calls in 0.000 seconds                               
                                                                                                          
                                                      Stats                                               
                                                                                                          
┏━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┓
┃ Function             ┃ GPU ncalls ┃ GPU cumtime ┃ GPU percall ┃ CPU ncalls ┃ CPU cumtime ┃ CPU percall ┃
┡━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━┩
│ Int64Index.__len__   │ 33         │ 0.010       │ 0.000       │ 0          │ 0.000       │ 0.000       │
│ Int64Index.set_names │ 3          │ 0.002       │ 0.001       │ 0          │ 0.000       │ 0.000       │
└──────────────────────┴────────────┴─────────────┴─────────────┴────────────┴─────────────┴─────────────┘

We can then read the created file with:


In [7]:
%%cudf.pandas.profile

xr.open_zarr("ds1.zarr", chunks=None)

<xarray.Dataset>
Dimensions:  (x: 4, y: 2, z: 6)
Coordinates:
  * x        (x) int64 0 1 2 3
  * y        (y) int64 -2 -1
  * z        (z) int64 -3 -2 -1 0 1 2
Data variables:
    a        (x, y) float64 ...
    b        (z, x) float64 ...

                                                                                                            
                                         Total time elapsed: 0.341 seconds                                  
                                      288 GPU function calls in 0.107 seconds                               
                                       10 CPU function calls in 0.005 seconds                               
                                                                                                            
                                                       Stats                                                
                                                                                                            
┏━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┓
┃ Function               ┃ GPU ncalls ┃ GPU cumtime ┃ GPU percall ┃ CPU ncalls ┃ CPU cumtime ┃ CPU percall ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━┩
│ isna                   │ 0          │ 0.000       │ 0.000       │ 10         │ 0.005       │ 0.001       │
│ Index                  │ 3          │ 0.003       │ 0.001       │ 0          │ 0.000       │ 0.000       │
│ Int64Index.__len__     │ 252        │ 0.074       │ 0.000       │ 0          │ 0.000       │ 0.000       │
│ Int64Index.copy        │ 6          │ 0.002       │ 0.000       │ 0          │ 0.000       │ 0.000       │
│ Int64Index.__setattr__ │ 6          │ 0.002       │ 0.000       │ 0          │ 0.000       │ 0.000       │
│ Int64Index.__getitem__ │ 18         │ 0.015       │ 0.001       │ 0          │ 0.000       │ 0.000       │
│ Int64Index.__repr__    │ 3          │ 0.011       │ 0.004       │ 0          │ 0.000       │ 0.000       │
└────────────────────────┴────────────┴─────────────┴─────────────┴────────────┴─────────────┴─────────────┘

Not all pandas operations ran on the GPU. The following functions required CPU fallback:

- isna

To request GPU support for any of these functions, please file a Github issue here: 
]8;id=348279;https://github.com/rapidsai/cudf/issues/new?assignees=&labels=%3F+-+Needs+Triage%2C+feature+request&projects=&template=pandas_function_request.md&title=%5BFEA%5D\https://github.com/rapidsai/cudf/issues/new/choose]8;;\.

setting the `chunks` parameter to `None` avoids `dask` (more on that in a later
session)


**tip:** You can write to any dictionary-like (`MutableMapping`) interface:

In [8]:
%%cudf.pandas.profile

mystore = {}

ds1.to_zarr(store=mystore)

                                                                                                          
                                        Total time elapsed: 0.096 seconds                                 
                                      36 GPU function calls in 0.011 seconds                              
                                      0 CPU function calls in 0.000 seconds                               
                                                                                                          
                                                      Stats                                               
                                                                                                          
┏━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┓
┃ Function             ┃ GPU ncalls ┃ GPU cumtime ┃ GPU percall ┃ CPU ncalls ┃ CPU cumtime ┃ CPU percall ┃
┡━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━┩
│ Int64Index.__len__   │ 33         │ 0.009       │ 0.000       │ 0          │ 0.000       │ 0.000       │
│ Int64Index.set_names │ 3          │ 0.002       │ 0.001       │ 0          │ 0.000       │ 0.000       │
└──────────────────────┴────────────┴─────────────┴─────────────┴────────────┴─────────────┴─────────────┘

## Raster files using rioxarray

[rioxarray](https://corteva.github.io/rioxarray/) is an *Xarray extension* that allows reading and writing a wide variety of geospatial image formats compatible with Geographic Information Systems (GIS), for example GeoTIFF.

If rioxarray is installed your environment it will be automatically detected and give you access to the `.rio` accessor:

In [9]:
%%cudf.pandas.profile

da = xr.DataArray(
    data=ds1.a.data,
    coords={
        "y": np.linspace(47.5, 47.8, 4),
        "x": np.linspace(-122.9, -122.7, 2),
    },
)

# Add Geospatial Coordinate Reference https://epsg.io/4326
# this is stored as a 'spatial_ref' coordinate
da.rio.write_crs("epsg:4326", inplace=True)
da

<xarray.DataArray (y: 4, x: 2)>
array([[ 1.76405235,  0.40015721],
       [ 0.97873798,  2.2408932 ],
       [ 1.86755799, -0.97727788],
       [ 0.95008842, -0.15135721]])
Coordinates:
  * y            (y) float64 47.5 47.6 47.7 47.8
  * x            (x) float64 -122.9 -122.7
    spatial_ref  int64 0

                                                                                                               
                                           Total time elapsed: 0.309 seconds                                   
                                        231 GPU function calls in 0.086 seconds                                
                                         0 CPU function calls in 0.000 seconds                                 
                                                                                                               
                                                         Stats                                                 
                                                                                                               
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┓
┃ Function                  ┃ GPU ncalls ┃ GPU cumtime ┃ GPU percall ┃ CPU ncalls ┃ CPU cumtime ┃ CPU percall ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━┩
│ Index                     │ 2          │ 0.005       │ 0.002       │ 0          │ 0.000       │ 0.000       │
│ Float64Index.__len__      │ 205        │ 0.059       │ 0.000       │ 0          │ 0.000       │ 0.000       │
│ Float64Index.copy         │ 4          │ 0.002       │ 0.000       │ 0          │ 0.000       │ 0.000       │
│ Float64Index.__setattr__  │ 4          │ 0.002       │ 0.000       │ 0          │ 0.000       │ 0.000       │
│ Float64Index.__deepcopy__ │ 2          │ 0.002       │ 0.001       │ 0          │ 0.000       │ 0.000       │
│ Float64Index.__getitem__  │ 12         │ 0.010       │ 0.001       │ 0          │ 0.000       │ 0.000       │
│ Float64Index.__repr__     │ 2          │ 0.007       │ 0.004       │ 0          │ 0.000       │ 0.000       │
└───────────────────────────┴────────────┴─────────────┴─────────────┴────────────┴─────────────┴─────────────┘

In [10]:
%%cudf.pandas.profile

da.rio.to_raster('ds1_a.tiff')

                                                                                                              
                                          Total time elapsed: 0.231 seconds                                   
                                        74 GPU function calls in 0.033 seconds                                
                                        0 CPU function calls in 0.000 seconds                                 
                                                                                                              
                                                        Stats                                                 
                                                                                                              
┏━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┓
┃ Function                 ┃ GPU ncalls ┃ GPU cumtime ┃ GPU percall ┃ CPU ncalls ┃ CPU cumtime ┃ CPU percall ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━┩
│ Float64Index.__len__     │ 46         │ 0.013       │ 0.000       │ 0          │ 0.000       │ 0.000       │
│ Float64Index.__getitem__ │ 24         │ 0.018       │ 0.001       │ 0          │ 0.000       │ 0.000       │
│ Float64Index.copy        │ 4          │ 0.002       │ 0.001       │ 0          │ 0.000       │ 0.000       │
└──────────────────────────┴────────────┴─────────────┴─────────────┴────────────┴─────────────┴─────────────┘

NOTE: you can now load this file into GIS tools like [QGIS](https://www.qgis.org)! Or open back into Xarray:

In [11]:
%%cudf.pandas.profile

DA = xr.open_dataarray('ds1_a.tiff', engine='rasterio')
DA.rio.crs

CRS.from_epsg(4326)

                                                                                                               
                                           Total time elapsed: 0.214 seconds                                   
                                        188 GPU function calls in 0.063 seconds                                
                                         0 CPU function calls in 0.000 seconds                                 
                                                                                                               
                                                         Stats                                                 
                                                                                                               
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┓
┃ Function                  ┃ GPU ncalls ┃ GPU cumtime ┃ GPU percall ┃ CPU ncalls ┃ CPU cumtime ┃ CPU percall ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━┩
│ Index                     │ 3          │ 0.005       │ 0.002       │ 0          │ 0.000       │ 0.000       │
│ Float64Index.__len__      │ 116        │ 0.035       │ 0.000       │ 0          │ 0.000       │ 0.000       │
│ Int64Index.__len__        │ 51         │ 0.016       │ 0.000       │ 0          │ 0.000       │ 0.000       │
│ Int64Index.copy           │ 3          │ 0.001       │ 0.000       │ 0          │ 0.000       │ 0.000       │
│ Int64Index.__setattr__    │ 2          │ 0.001       │ 0.000       │ 0          │ 0.000       │ 0.000       │
│ Float64Index.copy         │ 6          │ 0.002       │ 0.000       │ 0          │ 0.000       │ 0.000       │
│ Float64Index.__setattr__  │ 4          │ 0.002       │ 0.000       │ 0          │ 0.000       │ 0.000       │
│ Int64Index.__deepcopy__   │ 1          │ 0.001       │ 0.001       │ 0          │ 0.000       │ 0.000       │
│ Float64Index.__deepcopy__ │ 2          │ 0.002       │ 0.001       │ 0          │ 0.000       │ 0.000       │
└───────────────────────────┴────────────┴─────────────┴─────────────┴────────────┴─────────────┴─────────────┘